### About Dataset
 Context This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

 Content 5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619   entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset. Columns

 #### asin - ID of the product, like B000FA64PK
 #### helpful - helpfulness rating of the review - example: 2/3.
 #### overall - rating of the product.
 #### reviewText - text of the review (heading).
 #### reviewTime - time of the review (raw).
 #### reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
 #### reviewerName - name of the reviewer.
 #### summary - summary of the review (description).
 #### unixReviewTime - unix timestamp.
 #### Acknowledgements This dataset is taken from Amazon product data, Julian McAuley, UCSD website.  http://jmcauley.ucsd.edu/data/amazon/

 License to the data files belong to them.

 Inspiration

 Sentiment analysis on reviews.
 Understanding how people rate usefulness of a review/ What factors influence helpfulness of a review.
 Fake reviews/ outliers.
 Best rated product IDs, or similarity between products based on reviews alone (not the best idea ikr).
 Any other interesting analysis
### Best Practises
#### 1 Preprocessing And Cleaning
#### 2 Train Test Split
#### 3 BOW,TFIDF,Word2vec
#### 4 Train ML algorithms

In [46]:
import pandas as pd

In [47]:
df=pd.read_csv('all_kindle_review.csv')

In [48]:
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [49]:
## out of all features we work on review text and ratings
df=df[['reviewText','rating']]

In [50]:
df

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4
...,...,...
11995,Valentine cupid is a vampire- Jena and Ian ano...,4
11996,I have read all seven books in this series. Ap...,5
11997,This book really just wasn't my cuppa. The si...,3
11998,"tried to use it to charge my kindle, it didn't...",1


In [51]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [52]:
## if rating<3 then in the output it is negative rating else it is a positive rating 
## rating<3->0(negative rating)
## rating>=3 ->1(positive rating)
df['rating'] = df['rating'].apply(lambda x: 0 if x<3 else 1)


In [53]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


### Text preproceesing and cleaning

In [72]:
## convert the text into lower case
df['reviewText']=df['reviewText'].str.lower()

In [73]:
#### Removing special characters
#### Remove the stopswords
####   Remove url 
#### html tags
## Remove any additional spaces

In [94]:
import re
import nltk
from nltk.stem import WordNetLemmatizer
wd=WordNetLemmatizer()
nltk.download('stopwords')
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [95]:
#### Removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-zA-Z0-9]',' ',x))

### Remove the stopwords
df['reviewText'] = df['reviewText'].apply(lambda x:' '.join([word for word in x.split() if word not in stopwords.words('english')]))

## Remove url 
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))

## remove html
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

## remove additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))

In [103]:
def lemmatization(text):
    return " ".join([wd.lemmatize(word) for word in text.split()])

In [105]:
df['reviewText']=df['reviewText'].apply(lambda x:lemmatization(x))

In [106]:
df.head()

,reviewText,rating
0,jace rankin may short nothing mess man hauled ...,1
1,great short read want put read one sitting sex...,1
2,start saying first four book expecting 34 conc...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [147]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df['reviewText'],df['rating'],test_size=0.20,random_state=42)

In [148]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

bow=CountVectorizer(max_features=2500,ngram_range=(1,2))
tfidf=TfidfVectorizer(max_features=2500,ngram_range=(1,2))

In [149]:
X_train_bow=bow.fit_transform(X_train).toarray()
X_test_bow=bow.transform(X_test).toarray()

In [162]:
X_train_bow.shape

(9600, 2500)

In [163]:
X_test_bow.shape

(2400, 2500)

In [152]:
X_train_tfidf=tfidf.fit_transform(X_train).toarray()
X_test_tfidf=tfidf.transform(X_test).toarray()

In [153]:
from sklearn.naive_bayes import MultinomialNB
bow_model=MultinomialNB()
bow_model.fit(X_train_bow,y_train)
y_pred_bow=bow_model.predict(X_test_bow)

In [154]:
len(y_pred_bow)

2400

In [160]:
from sklearn.metrics import accuracy_score,confusion_matrix
score=accuracy_score(y_test,y_pred_bow)
print(score)
confusion_matrix(y_test,y_pred_bow)

0.8191666666666667


array([[ 458,  345],
       [  89, 1508]], dtype=int64)

In [159]:
tfidf_model=MultinomialNB()
tfidf_model.fit(X_train_tfidf,y_train)
y_pred_bow=tfidf_model.predict(X_test_tfidf)
score=accuracy_score(y_test,y_pred_bow)
print(score)

0.8191666666666667
